In [67]:
!pip install segmentation_models

     |████████████████████████████████| 50 kB 1.9 MB/s eta 0:00:01


In [1]:
import pandas as pd
import os, json
import time
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import random

In [ ]:
# from tensorflow.compat.v1 import ConfigProto
# from tensorflow.compat.v1 import InteractiveSession

# config = ConfigProto()
# config.gpu_options.allow_growth = True
# session = InteractiveSession(config=config)

In [ ]:
# data_folder = os.getcwd() + '/data_npfiles/image/img'
# patient_list = os.listdir(data_folder)
# len(patient_list)
# patient_list[0:5]
# patient_list = [patient.replace('_image.npy', '',1) for patient in patient_list]
# patient_list = [patient.replace('_mask.npy', '',1) for patient in patient_list]
# patient_list = list(set(patient_list))
# # patient_list.remove('.ipynb_checkpoints')
# patient_list = patient_list[0:50]
# len(patient_list)
# patient_list
# patient_list[596]

In [2]:
data_folder = os.getcwd() + '/data_npfiles/image/img'
patient_list = os.listdir(data_folder)

In [3]:
val_ratio = 0.2
SEED = 1
random.seed(SEED)
file_list = [i for i in range(len(patient_list))]
random.shuffle(file_list)
train_list = file_list[:int(len(patient_list)*(1-val_ratio))]
val_list = file_list[int(len(patient_list)*(1-val_ratio)):]
print(len(train_list),len(val_list))
val_list[0]

92704 23177


20918

In [10]:
import shutil
for i in val_list:
    old_image = 'data_npfiles/image/img/'+patient_list[i]
    new_image = 'data_npfiles/val_image/img/'+patient_list[i]
    
    old_mask = 'data_npfiles/mask/img/'+patient_list[i]
    new_mask = 'data_npfiles/val_mask/img/'+patient_list[i]
    
    shutil.move(old_image,new_image)
    shutil.move(old_mask,new_mask)
    
    # shutil.move(new_image,old_image)
    # shutil.move(new_mask,old_mask)

In [ ]:
# from skimage.transform import resize
# from skimage import img_as_bool
    
# train_images = []
# val_images = []
# train_masks = []
# val_masks = []

# for i in train_list:
#     image = np.load(data_folder+patient_list[i]+'_image.npy')
#     image = resize(image,(image.shape[0],256,256))
#     train_images.append(image)
    
#     mask = img_as_bool(np.load(data_folder+patient_list[i]+'_mask.npy'))
#     mask = img_as_bool(resize(mask,(mask.shape[0],256,256)))
#     train_masks.append(mask)

# for i in val_list:
#     image = np.load(data_folder+patient_list[i]+'_image.npy')
#     image = resize(image,(image.shape[0],256,256))
#     val_images.append(image)
    
#     mask = img_as_bool(np.load(data_folder+patient_list[i]+'_mask.npy'))
#     mask = img_as_bool(resize(mask,(mask.shape[0],256,256)))
#     val_masks.append(mask)

# train_images = np.vstack(train_images)
# val_images = np.vstack(val_images)
# train_masks = np.vstack(train_masks)
# val_masks = np.vstack(val_masks)

# print('train',train_images.shape,train_masks.shape,'test',val_images.shape,val_masks.shape)

In [ ]:
# f, a = plt.subplots(1,2)
# s = 186
# a[0].imshow(test_images[s]/255.0, cmap="gray")
# a[1].imshow(test_masks[s]/255.0)

In [ ]:
# import tensorflow as tf
# width = 512
# height = 512
# train_images_processed = train_images.reshape(train_images.shape[0], width, height, 1)/255.0
# test_images_processed = test_images.reshape(test_images.shape[0], width, height, 1)/255.0
# train_masks_processed = tf.one_hot(train_masks,2)
# test_masks_processed = tf.one_hot(test_masks,2)
# # array = tf.one_hot(test_masks,2)
# print('train',train_images_processed.shape,test_images_processed.shape,'test',train_masks_processed.shape,test_masks_processed.shape)

In [3]:
import keras.backend as K
def dice_coef(y_true, y_pred, smooth=1):
    intersection = K.sum(y_true * y_pred, axis=[1,2,3])
    union = K.sum(y_true, axis=[1,2,3]) + K.sum(y_pred, axis=[1,2,3])

    return K.mean( (2. * intersection + smooth) / (union + smooth), axis=0)

def dice_coef_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)

In [4]:
img_size = (256,256)
num_classes = 2

In [5]:
from tensorflow import keras
from tensorflow.keras import layers


def get_model(img_size, num_classes):
    inputs = keras.Input(shape=img_size + (1,))

    ### [First half of the network: downsampling inputs] ###

    # Entry block
    x = layers.Conv2D(32, 3, strides=2, padding="same")(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    previous_block_activation = x  # Set aside residual

    # Blocks 1, 2, 3 are identical apart from the feature depth.
    for filters in [64, 128, 256]:
        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(filters, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(filters, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.MaxPooling2D(3, strides=2, padding="same")(x)

        # Project residual
        residual = layers.Conv2D(filters, 1, strides=2, padding="same")(
            previous_block_activation
        )
        x = layers.add([x, residual])  # Add back residual
        previous_block_activation = x  # Set aside next residual

    ### [Second half of the network: upsampling inputs] ###

    for filters in [256, 128, 64, 32]:
        x = layers.Activation("relu")(x)
        x = layers.Conv2DTranspose(filters, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.Activation("relu")(x)
        x = layers.Conv2DTranspose(filters, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.UpSampling2D(2)(x)

        # Project residual
        residual = layers.UpSampling2D(2)(previous_block_activation)
        residual = layers.Conv2D(filters, 1, padding="same")(residual)
        x = layers.add([x, residual])  # Add back residual
        previous_block_activation = x  # Set aside next residual

    # Add a per-pixel classification layer
    # outputs = layers.Conv2D(num_classes, 3, activation="softmax", padding="same")(x)
    outputs = layers.Conv2D(1, 1, activation="sigmoid", padding="same")(x)
    
    # Define the model
    model = keras.Model(inputs, outputs)
    return model


# Free up RAM in case the model definition cells were run multiple times
keras.backend.clear_session()

# Build model
model = get_model(img_size, num_classes)
model.summary()

2021-10-02 05:46:21.787663: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-10-02 05:46:21.883692: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-10-02 05:46:21.884296: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-10-02 05:46:21.885435: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 1) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 128, 128, 32) 320         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 128, 128, 32) 128         conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 128, 128, 32) 0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [167]:
import tensorflow as tf
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Dropout,UpSampling2D,concatenate

def unet(pretrained_weights = None,input_size = (512,512,1)):
    inputs = tf.keras.Input(shape=input_size)
    conv1 = Conv2D(64, 3, activation = 'relu', dilation_rate=2,padding = 'same', kernel_initializer = 'he_normal')(inputs)
    conv1 = BatchNormalization()(conv1)
    conv1 = Conv2D(64, 3, activation = 'relu', dilation_rate=2,padding = 'same', kernel_initializer = 'he_normal')(conv1)
    conv1 = BatchNormalization()(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = Conv2D(128, 3, activation = 'relu', dilation_rate=2,padding = 'same', kernel_initializer = 'he_normal')(pool1)
    conv2 = BatchNormalization()(conv2)
    conv2 = Conv2D(128, 3, activation = 'relu', dilation_rate=2, padding = 'same', kernel_initializer = 'he_normal')(conv2)
    conv2 = BatchNormalization()(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
    conv3 = BatchNormalization()(conv3)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
    conv3 = BatchNormalization()(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
    conv4 = BatchNormalization()(conv4)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
    conv4 = BatchNormalization()(conv4)
    drop4 = Dropout(0.5)(conv4, training=True)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
    conv5 = BatchNormalization()(conv5)
    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
    conv5 = BatchNormalization()(conv5)
    drop5 = Dropout(0.5)(conv5, training=True)

    up6 = Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))
    merge6 = concatenate([drop4,up6], axis = 3)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)
    

    up7 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
    merge7 = concatenate([conv3,up7], axis = 3)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)
    

    up8 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
    merge8 = concatenate([conv2,up8], axis = 3)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)
    

    up9 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
    merge9 = concatenate([conv1,up9], axis = 3)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv9 = Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
   
    conv10 = Conv2D(1, 1, activation = 'sigmoid')(conv9)

    model = tf.keras.Model(inputs = inputs, outputs = conv10)

    model.compile(optimizer = keras.optimizers.Adam(lr = 1e-4), loss = 'binary_crossentropy', metrics = 'accuracy')
    

    if(pretrained_weights):
        model=keras.models.load_model(pretrained_weights)

    return model

model2 = unet()
model2.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 512, 512, 1) 0                                            
__________________________________________________________________________________________________
conv2d_9 (Conv2D)               (None, 512, 512, 64) 640         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_15 (BatchNo (None, 512, 512, 64) 256         conv2d_9[0][0]                   
__________________________________________________________________________________________________
conv2d_10 (Conv2D)              (None, 512, 512, 64) 36928       batch_normalization_15[0][0]     
____________________________________________________________________________________________

In [6]:
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
seed = 100 # (IMPORTANT) to transform image and corresponding mask with same augmentation parameter.

train_image_datagen = ImageDataGenerator() # custom fuction for each image you can use resnet one too.
train_mask_datagen = ImageDataGenerator()  # to make mask as feedable formate (256,256,1)

val_image_datagen = ImageDataGenerator()
val_image_datagen = ImageDataGenerator()

train_image_generator = train_image_datagen.flow_from_directory("data_npfiles/image/",
                                                                class_mode=None, 
                                                                seed=seed, 
                                                                color_mode='grayscale',
                                                                # target_size=(512, 512),
                                                                batch_size=64,
                                                                # subset = 'training'
                                                               )
train_mask_generator = train_mask_datagen.flow_from_directory("data_npfiles/mask/",
                                                              class_mode=None,
                                                              seed=seed, 
                                                              color_mode='grayscale',
                                                              # target_size=(512, 512),
                                                              batch_size=64,
                                                              # subset='training'
                                                             )

val_image_generator = train_image_datagen.flow_from_directory("data_npfiles/val_image/",
                                                            class_mode=None, 
                                                            seed=seed, 
                                                            color_mode='grayscale',
                                                            batch_size=32,
                                                            # subset = 'validation'
                                                             )
val_mask_generator = val_image_datagen.flow_from_directory("data_npfiles/val_mask/",
                                                            class_mode=None, 
                                                            seed=seed, 
                                                            color_mode='grayscale',
                                                            batch_size=32,
                                                            # subset = 'validation',
                                                          )

train_generator = zip(train_image_generator, train_mask_generator)
val_generator = zip(val_image_generator, val_mask_generator)

Found 172503 images belonging to 2 classes.
Found 172503 images belonging to 2 classes.
Found 0 images belonging to 2 classes.
Found 0 images belonging to 2 classes.


In [7]:
def my_image_mask_generator(image_data_generator, mask_data_generator):
    train_generator = zip(image_data_generator, mask_data_generator)
    for (img, mask) in train_generator:
        yield (img/255.0, mask/255.0)
        # print(np.max(img))
my_generator = my_image_mask_generator(train_image_generator, train_mask_generator)

my_val_generator = my_image_mask_generator(val_image_generator, val_mask_generator)

In [ ]:
opt = keras.optimizers.Adam(learning_rate=1e-4)
model.compile(optimizer=opt, loss=['binary_crossentropy'], metrics=[dice_coef])
callbacks = [
    keras.callbacks.ModelCheckpoint("model/lung_segmentation.h5", save_best_only=True)
]
epochs = 10
history = model.fit_generator(my_generator , epochs=epochs,steps_per_epoch=300,validation_steps =50)
# history = model2.fit_generator(my_generator , epochs=epochs,steps_per_epoch=100, validation_data=val_generator,validation_steps =50)

Epoch 1/10
300/300 [==============================] - 648s 2s/step - loss: 9.5525e-05 - dice_coef: 0.7030
Epoch 2/10
300/300 [==============================] - 641s 2s/step - loss: 9.2271e-05 - dice_coef: 0.7190
Epoch 3/10
300/300 [==============================] - 703s 2s/step - loss: 8.7557e-05 - dice_coef: 0.7335
Epoch 4/10
300/300 [==============================] - 723s 2s/step - loss: 8.6219e-05 - dice_coef: 0.7319
Epoch 5/10
300/300 [==============================] - 722s 2s/step - loss: 9.1683e-05 - dice_coef: 0.7248
Epoch 6/10
300/300 [==============================] - 728s 2s/step - loss: 8.7908e-05 - dice_coef: 0.7256
Epoch 7/10
 83/300 [=======>......................] - ETA: 8:32 - loss: 8.4650e-05 - dice_coef: 0.7466

In [ ]:
# Get training and test loss histories
training_loss = history.history['loss']
# test_loss = history.history['val_loss']

# Create count of the number of epochs
epoch_count = range(1, len(training_loss) + 1)

# Visualize loss history
plt.plot(epoch_count, training_loss, 'r--')
# plt.plot(epoch_count, test_loss, 'b-')
plt.legend(['Training Loss']) #, 'Test Loss']
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.show()

In [ ]:
from PIL import Image
from skimage.transform import resize
def print_pred(model):
    im_img = plt.imread('data_npfiles/image/img/LIDC-IDRI-0002_90.tif')
    im_mask = plt.imread('data_npfiles/mask/img/LIDC-IDRI-0002_90.tif')
    im_pred = model.predict(resize(im_img.reshape(1,512,512,1),(1,256,256,1)))

    plt.figure()
    f, axarr = plt.subplots(1,3) 

    axarr[0].imshow(im_img/255.0, cmap='Greys_r')
    axarr[1].imshow(im_mask/255.0, cmap='Greys_r')
    axarr[2].imshow(im_pred[0,:,:,0], cmap='Greys_r')
print_pred(model)

In [ ]:
patients = ['LIDC-IDRI-0298','LIDC-IDRI-0300','LIDC-IDRI-0314','LIDC-IDRI-0325','LIDC-IDRI-0345',
           'LIDC-IDRI-0069','LIDC-IDRI-0084','LIDC-IDRI-0094','LIDC-IDRI-0095','LIDC-IDRI-0106',
           'LIDC-IDRI-0078','LIDC-IDRI-0079','LIDC-IDRI-0086','LIDC-IDRI-0098','LIDC-IDRI-0101']
